In [1]:
import json
import numpy as np
import os
import pandas as pd
with open("file_paths.json", "r") as fp:
    filepaths = json.load(fp)
    
def conversational_turns(sheet):
    """
    Function to count conversational turns in a 
    human-coded spreadsheet using the LENA Foundation
    definition, "(back and forth interactions between 
    a child and adult) are computed by counting the 
    number of times utterances spoken by the child or 
    adult were responded to by another person."
    
    Parameters
    ----------
    sheet: DataFrame
    
    Returns
    -------
    turn_count: int
    
    Example
    -------
    >>> import numpy as np
    >>> import pandas as pd
    >>> conversational_turns(
    ...     pd.DataFrame({
    ...         "Child": [
    ...             "spontaneous speech",
    ...             np.nan,
    ...             "no response",
    ...             np.nan,
    ...             "verbal response"
    ...         ],
    ...         "Parent": [
    ...             np.nan,
    ...             "neutral talk",
    ...             np.nan,
    ...             "open-ended question",
    ...             np.nan
    ...         ]
    ...     })
    ... )
    2
    """
    turn_count = 0
    half_turn = {
        "Child": False,
        "Parent": False,
        "Stranger ": False
    }
    stranger = sheet[
        "Stranger "
    ] if "Stranger " in sheet.columns else pd.Series(
        sheet["Child"].apply(
            lambda x: np.nan
        )
    )
    for i in range(
        len(
            sheet["Child"]
        )
    ):
        if sheet.loc[i, "Child"] in vocalizations["child"]:
            if half_turn["Parent"] or half_turn["Stranger "]:
                turn_count += 1
                half_turn["Parent"] = half_turn["Stranger "] = False
            half_turn["Child"] = True
        if sheet.loc[i, "Parent"] in vocalizations["adult"]:
            if half_turn["Child"] or half_turn["Stranger "]:
                turn_count += 1
                half_turn["Child"] = half_turn["Stranger "] = False
            half_turn["Parent"] = True
        if stranger[i] in vocalizations["adult"]:
            if half_turn["Child"] or half_turn["Parent"]:
                turn_count += 1
                half_turn["Child"] = half_turn["Stranger "] = False
            half_turn["Stranger "] = True
        if (
            sheet.loc[i, "Child"] not in vocalizations["child"]
        ) and (
            sheet.loc[i, "Parent"] not in vocalizations["adult"]
        ) and (
            stranger[i] not in vocalizations["adult"]
        ):
            half_turn["Child"] = half_turn["Parent"] = half_turn["Stranger "] = False
    return(turn_count)

def pearson_coder_format(
    human_ratings,
    LENA_ratings,
    measure,
    ursis
):
    """
    Function to take data from Multi-index DataFrames 
    and convert those data to the format
    x : array_like
        A 1-D array containing multiple variables
        and observations. Each row of x represents a
        variable, and each column a single observation
        of all those variables.
    y : array_like, optional
        An additional set of variables and observations.
        y has the same shape as x.

    Parameters
    ----------
    human_ratings: DataFrame
        Multi-index DataFrame (indices = URSI, session,
        file, in that sequence)
    
    LENA_ratings: DataFrame
        Multi-index DataFrame (indices = URSI, session,
        in that sequence)

    measure: string
        column header to collect
        
    ursis: iterable
        iterable of values for first index
        
    Returns
    -------
    formatted: dictionary
        key: string
            ursi
        value: list
            [x, y]
            x: array
            y: array
    """
    return(
        {
            ursi: [
                np.array([
                    np.mean(
                        [
                            (
                                human_ratings.loc[
                                    (ursi, "A1", f),
                                    measure
                                ] + human_ratings.loc[
                                    (ursi, "A2", f),
                                    measure
                                ] + human_ratings.loc[
                                    (ursi, "A3", f),
                                    measure
                                ]
                            ) for f in list(
                                human_ratings.loc[
                                    (ursi, "B"),
                                ].index
                            )
                        ]
                    ),
                    np.mean([
                        human_ratings.loc[
                            (ursi, "B", f),
                            measure
                        ] for f in list(
                            human_ratings.loc[
                                (ursi, "B"),
                            ].index)
                    ]),
                    np.mean([
                        human_ratings.loc[
                            (ursi, "C", f),
                            measure
                        ] for f in list(
                            human_ratings.loc[
                                (ursi, "C"),
                            ].index)
                    ])
                ]),
            np.array([
                    LENA_ratings.loc[
                        (ursi, "A"),
                        measure
                    ],
                    LENA_ratings.loc[
                        (ursi, "B"),
                        measure
                    ],
                    LENA_ratings.loc[
                        (ursi, "C"),
                        measure
                    ]
                ])
            ] for ursi in ursis
        }
    )

sessions = {
    "A1",
    "B",
    "A2",
    "C",
    "A3"
}

vocalizations = {
    "child": {
        "verbal response",
        "noise",
        "spontaneous speech"
    },
    "adult": {
        "open-ended question",
        "forced choice question",
        "yes/no question",
        "command",
        "neutral talk"
    }
}

In [2]:
LENA = pd.read_csv(
    os.path.join(
        filepaths["LENA"],
        "CPP_data.csv"
    )
)
LENA["URSI"] = LENA["URSI"].apply(
    lambda u: u.replace(
        "M04",
        "M004"
    )
)
ursis = set(LENA["URSI"].unique())
humans = {
    fp: {
        "ursi": fp[:9],
        **{sheetname: pd.read_excel(
            os.path.join(
                filepaths["humans"],
                fp
            ),
            sheet_name=sheetname
        ) for sheetname in sessions}
    } for fp in os.listdir(
        filepaths["humans"]
    ) if fp[:9] in ursis
}

In [3]:
human_ratings = pd.DataFrame({
    (ursi, session, f): {
        "Turn_Count": conversational_turns(
            humans[
                f
            ][
                session
            ]
        ),
        "Child_Voc_Count": len(
            [
                v for v in humans[
                    f
                ][
                    session
                ][
                    "Child"
                ] if v in vocalizations["child"]
            ]
        )
    } for ursi in ursis for session in sessions for f in humans if f.startswith(ursi) 
}).T.sort_index()
LENA_ratings = LENA.groupby(["URSI", "Session"]).sum()[["Turn_Count","Child_Voc_Count"]]

In [4]:
voc_counts = pearson_coder_format(
    human_ratings,
    LENA_ratings,
    "Child_Voc_Count",
    ursis
)
turns = pearson_coder_format(
    human_ratings,
    LENA_ratings,
    "Turn_Count",
    ursis
)
r = pd.DataFrame({
    ursi: {
        ("Child_Voc_Count", "Pearson's r"): np.corrcoef(
            *voc_counts[ursi]
        )[1,0],
        ("Turn_Count", "Pearson's r"): np.corrcoef(
            *turns[ursi]
        )[1,0]
    } for ursi in ursis
}).T

In [ ]:
r

In [6]:
for col in r.columns:
    print(r[col].describe())

count    24.000000
mean      0.897176
std       0.291507
min      -0.414118
25%       0.958501
50%       0.980533
75%       0.996660
max       0.999950
Name: (Child_Voc_Count, Pearson's r), dtype: float64
count    24.000000
mean      0.813401
std       0.353978
min      -0.352801
25%       0.803529
50%       0.978106
75%       0.998229
max       0.999964
Name: (Turn_Count, Pearson's r), dtype: float64
